## 324 - Build a Custom Local Data Cache with Python

[Youtube](https://www.youtube.com/watch?v=Pl47QBjPlu4)

In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
from siphon.simplewebservice.wyoming import WyomingUpperAir

In [2]:
def get_filename(date: datetime, station: str) -> str:
    """
    Create a filename in the format YYYYMMDDHH_STN.txt
    """
    return f"{date.strftime('%Y%m%d%H')}_{station}.txt"

In [3]:
def save_to_cache(df: pd.DataFrame, file_path: Path):
    """
    Saves the pandas DataFrame to a text file CSV.
    """
    df.to_csv(file_path, index = False)
    print(f'Data saved to cache: {file_path}')

In [4]:
def load_from_cache(file_path: Path) -> pd.DataFrame:
    """
    Load a pandas DataFrame from the saved cache CSV file.
    """
    print(f'Loading from cache: {file_path}')
    return pd.read_csv(file_path)

In [5]:
def get_sounding_data(date: datetime, station: str, cache_dir: Path) -> pd.DataFrame:
    """
    Get sounding data from cache or remote source. If remote, store it locally.
    """
    # Make sure cache exists or create it
    if not cache_dir.exists():
        cache_dir.mkdir(parents=True, exist_ok=True)
        print(f'Created cache directory: {cache_dir}')

    filename = get_filename(date, station)
    file_path = cache_dir / filename

    if file_path.exists():
        print('Loading Existing')
        df = load_from_cache(file_path)
    else:
        print('Downloading new data')
        df = WyomingUpperAir.request_data(date, station)
        save_to_cache(df, file_path)
    return df

In [6]:
cache_directory = Path('sounding_data')

In [7]:
df_sounding = get_sounding_data(datetime(2014, 11, 1, 12), 'OUN', cache_directory)

Loading Existing
Loading from cache: sounding_data/2014110112_OUN.txt


In [8]:
df_sounding.head()

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station,station_number,time,latitude,longitude,elevation,pw
0,988.0,345,1.4,-4.6,90.0,5.0,-5.000000,-3.061617e-16,OUN,72357,2014-11-01 12:00:00,35.18,-97.44,345.0,7.29
1,974.0,461,1.6,-8.4,92.0,13.0,-12.992081,4.536935e-01,OUN,72357,2014-11-01 12:00:00,35.18,-97.44,345.0,7.29
2,964.0,544,2.6,-11.4,94.0,19.0,-18.953717,1.325373e+00,OUN,72357,2014-11-01 12:00:00,35.18,-97.44,345.0,7.29
3,956.2,610,2.4,-12.0,95.0,24.0,-23.908673,2.091738e+00,OUN,72357,2014-11-01 12:00:00,35.18,-97.44,345.0,7.29
4,945.0,706,2.2,-12.8,106.0,20.0,-19.225234,5.512747e+00,OUN,72357,2014-11-01 12:00:00,35.18,-97.44,345.0,7.29
